<a href="https://colab.research.google.com/github/ViktorBakken/SM-r/blob/main/models/Logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>